In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
data = pd.read_csv('./data/fu2405_tick.csv')

# data = data[-25000:]
# data.tail()

In [3]:
def argmax(x):
    return x.argmax()
def argmin(x):
    return x.argmin()
def get_diff(x):
    if abs(x[0])>abs(x[1]):
        return x[0]
    else:
        return x[1]
data['volume_delta'] = data['volume'].diff()

In [4]:
p0 = 60
p1 = 240

roll_0 = data['last_price'].rolling(p0)
roll_1 = data['last_price'].rolling(p1)

max0 = roll_0.max()
min0 = roll_0.min()

max1 = roll_1.max()
min1 = roll_1.min()

data['min0_diff'] = data['last_price'] - max0
data['max0_diff'] = data['last_price'] - min0

data['min1_diff'] = data['last_price'] - max1 
data['max1_diff'] = data['last_price'] - min1

data['diff0']= data[['min0_diff','max0_diff']].apply(get_diff,axis=1)
data['diff1']= data[['min1_diff','max1_diff']].apply(get_diff,axis=1)

In [5]:
#long_benifit = 6, long_loss = -9 ,short_benifit = 7, short_loss = -11
# long盈利41，交易次数4， short盈利18，交易次数10 
h1 = data[(data['last_price']<=data['bid_price'])&(data['bid_volume']>2*data['ask_volume'])&(data['volume_delta']>=10)].index
l1 = data[(data['last_price']>=data['ask_price'])&(data['ask_volume']>2*data['bid_volume'])&(data['volume_delta']<=10)].index

l1_close = data[(data['bid_volume']>2*data['ask_volume'])&(data['volume_delta']>=10)].index
h1_close = data[(data['ask_volume']>2*data['bid_volume'])&(data['volume_delta']<=10)].index

In [12]:
data['sign']=0
data['sign'][h1] = 1
data['sign'][l1] = -1

data['close_sign'] = 0
data['close_sign'][l1_close] = 1
data['close_sign'][h1_close] = -1

test_data = data[-790000:-560000]

C:\Users\rcl\AppData\Local\Temp\ipykernel_1052\3252150563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sign'][h1] = 1
C:\Users\rcl\AppData\Local\Temp\ipykernel_1052\3252150563.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sign'][l1] = -1
C:\Users\rcl\AppData\Local\Temp\ipykernel_1052\3252150563.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['close_sign'][l1_close] = 1
C:\Users\rcl\AppData\Local\Temp\i

In [13]:
pos = 0

long_benifit = 22
long_loss = -24
short_benifit = 22
short_loss = -24

long_diff = []
short_diff = []
total_diff = []
direction = []
pos_price = 0


for row in tqdm(test_data.iterrows()):
    sign = row[1]['sign']  
    if len(total_diff)>=3:
        if direction[-1] == direction[-2] == -1:
            if total_diff[-1]<0 and total_diff[-2]<0:
                sign*=-1
        elif total_diff[-1]<0 and total_diff[-2]<0 and total_diff[-3]<0:
            sign*= -1
            
    if pos == 0:
        if sign == 1 and row[1]['ask_volume']>0:
            pos_price = row[1]['ask_price']
            pos = 1
            direction.append(1)
            
        elif sign == -1 and row[1]['bid_volume']>0:
            pos_price = row[1]['bid_price']
            pos = -1
            direction.append(-1)
            
    elif pos == 1 and row[1]['bid_price']>0:
        diff = row[1]['bid_price'] - pos_price
        if diff>=long_benifit and row[1]['close_sign'] == -1:
            long_diff.append(diff)
            pos = 0
            total_diff.append(diff)
        elif diff<= long_loss:
            long_diff.append(diff)
            pos = 0
            total_diff.append(diff)
    elif pos == -1 and row[1]['ask_price']>0:
        diff = pos_price - row[1]['ask_price']
        if diff>=short_benifit and row[1]['close_sign'] == 1:
            short_diff.append(diff)
            pos = 0
            total_diff.append(diff)
        elif diff<= short_loss:
            short_diff.append(diff)
            pos = 0
            total_diff.append(diff)
        
print(sum(long_diff),len(long_diff))
print(sum(short_diff),len(short_diff))

230000it [00:17, 13281.51it/s]

43.0 7
42.0 4


In [9]:
for i,j in zip(direction,total_diff):
    print(i,j)

1 23.0
1 22.0
-1 22.0
1 24.0
1 24.0
1 23.0
1 22.0
1 -30.0
1 -24.0
1 27.0
-1 23.0
-1 -31.0
1 44.0
1 22.0
-1 23.0
1 -24.0
1 -24.0
1 22.0
-1 -24.0
1 28.0
1 22.0
1 27.0
-1 22.0
-1 -24.0
1 -24.0
1 22.0
